In [1]:
#import libraries
import pandas as pd
import tensorflow as tf
from keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
import random
from sklearn import metrics
import time


In [2]:
dnames = ['ddaccgz_10', 'ddaccgz_20', 'ddaccgz_30', 'ddaccgz_40', 'ddaccgz_50', 'ddaccgz_60', 'ddaccgz_70', 'ddaccgz_80', 'ddaccgz_90', 'ddaccgz_100']
dnames = ['ddaccgz_50']
tle = []

for a in range(len(dnames)):
    x = np.load('../data/training_data/' + dnames[a] + '_input_x.npy')
    y = np.load('../data/training_data/' + dnames[a] + '_input_y.npy')

    result = np.where(y==1) # Identify take over instances
    all_indices = list(range(y.shape[0]))
    ntc_index = [i for i in all_indices if i not in result[0]] # Identify non-take over instances
    tc_index = list(result[0])

    # Take a subset of the non-take over instances such that class ratio is 1:1
    tc_idx_size = len(tc_index)
    random.shuffle(ntc_index)
    ntc_data_idx = ntc_index[:tc_idx_size]

    # Combine and shuffle the indices
    idx = ntc_data_idx+tc_index
    random.shuffle(idx)

    X = x[idx]
    Y = y[idx]
    Y = Y.reshape(-1,1)
    
    tf.random.set_seed(0)

    model = models.Sequential()
    model.add(layers.Conv2D(8, (3,3), activation='relu', input_shape = (X.shape[1],X.shape[2],1)))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(16, (3,3), activation='relu'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(16, activation='tanh'))
    model.add(layers.Dense(8, activation='tanh'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    met = []
    for i in range(10):
        x_t, x_test, y_t, y_test = train_test_split(X, Y, test_size=0.25, shuffle=True)
        x_train, x_val, y_train, y_val = train_test_split(x_t, y_t, test_size=0.25, shuffle=True)

        start = time.time()

        history = model.fit(x_train, y_train, epochs=20, validation_data=(x_val, y_val), verbose=0)

        end = time.time()

        y_pred = model.predict(x_test, verbose=0)
        for i in y_pred:
            if i[0] >= 0.5:
                i[0] = 1
            else:
                i[0] = 0

        cf = metrics.confusion_matrix(y_test, y_pred)
        total=sum(sum(cf))
        accuracy = (cf[0,0]+cf[1,1])/total
        specificity = cf[0,0]/(cf[0,0]+cf[0,1])
        sensitivity = cf[1,1]/(cf[1,0]+cf[1,1]) #recall
        precision = cf[1,1]/(cf[0,1]+cf[1,1])
        f1 = 2*(1/((1/precision)+(1/sensitivity)))
        ti = end - start

        lt = [accuracy, specificity, sensitivity, precision, f1, ti]
        met.append(lt)

    metric = pd.DataFrame(met, columns=['accuracy', 'specificity', 'sensitivity', 'precision', 'f1-score', 'train_time'])

    f_accuracy = metric['accuracy'].mean()
    f_specificity = metric['specificity'].mean()
    f_sensitivity = metric['sensitivity'].mean() #recall
    f_precision = metric['precision'].mean()
    f_f1 = metric['f1-score'].mean()
    f_time = metric['train_time'].mean()

    tp = [dnames[a], f_accuracy, f_specificity, f_sensitivity, f_precision, f_f1, f_time]
    tle.append(tp)

table = pd.DataFrame(tle, columns=['datatype', 'accuracy', 'specificity', 'sensitivity', 'precision', 'f1-score', 'train_time'])
table.to_excel('../data/results/accuracy_tables/ddaccgz_kfold.xlsx', index=False)


Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-26 17:44:55.290191: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-26 17:44:55.290304: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-02-26 17:44:55.672424: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-26 17:44:55.830113: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 17:44:58.170678: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 17:45:41.998056: I tensorflow/core/grappler/o